In [ ]:
# 12.1 DeepDream模型
# （1）下载预训练模型
# 下载预训练模型vgg19
vgg = models.vgg19(pretrained=True)
vgg = vgg.to(device)
print(vgg)
modulelist = list(vgg.features.modules())

In [1]:
# (2)函数prod的主要功能
def prod(image, layer, iterations, lr):
    input = preprocess(image).unsqueeze(0)
    input = input.to(device).requires_grad_(True)
    vgg.zero_grad()
    for i in range(iterations):
        out = input
        for j in range(layer):
            out = modulelist[j+1](out)
        # 以特征值的L2为损失值
        loss = out.norm()
        loss.backward()
        # 使梯度增大
        with torch.no_grad():
            input += lr*input.grad
        input = input.squeeze()
        # 交互维度
        input.transpose_(0,1)
        input.transpose_(1,2)
        # 使数据限制在[0,1]之间
        input = np.clip(deprocess(input).detach().cpu().numpy(),0,1)
        im = Image.fromarray(np.uint8(input*255))
        return im

In [ ]:
# (3)函数deep_dream_vgg的主要功能
# 函数deep_dream_vgg是一个递归函数，多次缩小图像，然后调用函数prod。
# 接着在放大结果，并与按一定比例图像混合在一起，最终得到与输入图像相同大小的输出图像
def deep_dream_vgg(image,layer,iterations,lr,octave_scale=2,num_octaves=20):
    if num_octaves>0:
        image1 = image.filter(ImageFilter.GaussianBlur(2))
        if(image1.size[0]/octave_scale<1 or image1.size[1]/octave_scale<1):
            size = image1.size
        else:
            size = (int(image1.size[0]/octave_scale), int(image1.size[1]/octave_scale))
        # 缩小图像
        image1 = image1.resize(size, Image.ANTIALIAS)
        image1 = deep_dream_vgg(image1, layer, iterations, lr, octave_scale, num_octaves-1)
        size = (image.size[0], image.size[1])
        # 放大图像
        image1 = image1.resize(size, Image.ANTIALIAS)
        image = ImageChops.blend(image, image1, 0.6)
    img_result = prod(image, layer, iterations, lr)
    img_result = img_result.resize(image.size)
    plt.imshow(img_result)
    return img_result

In [ ]:
# (4)运行结果
# 输入图像
night_sky = load_image('data/starry_night.jpg')
# 使用VGG19的第4层
night_sky_4 = deep_dream_vgg(night_sky, 4, 6, 0.2)
# 使用VGG19的第8层
night_sky_8 = deep_dream_vgg(night_sky, 8, 6, 0.2)


In [ ]:
# 12.2 风格迁移
# 实现风格迁移核心思想就是定义损失函数，如何定义损失函数就成为解决问题的关键。
# 这个损失函数应该包括内容损失和风格损失，这里的损失包括风格损失和内容损失
loss = distance(style(reference_image) - style(generated_image)) + distance(content(original_image)-content(generated_image))

In [1]:
# 12.2.1 内容损失
# （1）定义内容损失函数
class ContentLoss(nn.Module):
    def __init__(self, target,):
        # 必须要用detach来分离出target，这时target不再是一个Variable
        # 这是为了动态计算梯度， 否则forward会出错，不能向前传播
        self.target = target.detach()
    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

NameError: name 'nn' is not defined

In [ ]:
# (2) 在卷积层上求损失值
content_layers = ['conv_4']
if name in content_layers:
    # 累加内容损失
    target = model(content_img).detach()
    content_loss = ContentLoss(target)
    model.add_module("content_loss_{}".format(i), content_loss)
    content_losses.append(content_loss)

In [ ]:
# 12.2.2 风格损失
# （1）先计算格拉姆矩阵
def gram_matrix(input):
    a,b,c,d = input.size()
    # a表示batch_size的大小
    # b是特征图的数量
    # （c,d）是特征图的维度（N=c*d）
    features = input.view(a*b, c*d)
    G = torch.mm(features, features.t()) # 计算内积
    # 对格拉姆矩阵标准化
    # 通过对其除以特征图像素总数
    return G.div(a*b*c*d)

In [ ]:
# （2）计算风格损失
class StyleLoss(nn.Module):
    def __init__(self, target_features):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()
    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

In [ ]:
# (3)在多个卷积层的累加
style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
if name in style_layers:
    # 累加风格损失
    target_feature = model(style_img).detach()
    style_loss = StyleLoss(target_feature)
    model.add_module("style_loss_{}".format(i), style_loss)
    style_losses.append(style_loss)

In [ ]:
# (4)总损失
# 在计算总的损失值时，对内容损失和风格损失是有侧重的，即需要对各自损失值加上权重
for sl in style_losses:
    style_score += sl.loss
    for cl in content_losses:
        content_score += cl.loss
    style_score *= style_weight
    content_score *= content_weight
    loss = style_score + content_score

In [ ]:
# 12.2.3 用PyTorch实现神经网络风格迁移
